In [1]:
import numpy as np
import tensorflow as tf
import time
from scipy import misc
import pickle
import scipy.optimize as sopt
import matplotlib.pyplot as plt
from IPython.display import clear_output as clr
np.random.seed(3038)

In [2]:
def load_cifar10(directory):
    print('Loading cifar10 Dataset...')
    X_tr = []
    X_te = None
    Y_tr = []
    Y_te = None

    for i in range(5):
        with open(directory+'/data_batch_' + str(i+1), 'rb') as file:
            data_tr = pickle.load(file, encoding='latin1')
            X_tr.append(np.reshape(data_tr['data'], [10000,3,32,32]))
            Y_tr.append(np.array(data_tr['labels']))

    with open(directory+'/test_batch', 'rb') as file:
        data_te = pickle.load(file, encoding='latin1')
        X_te = np.reshape(data_te['data'],[10000,3,32,32])
        Y_te = np.array(data_te['labels'])

    X = np.concatenate(X_tr, axis = 0).astype(np.float)
    X = np.swapaxes(X, 1, 3)
    X_train = np.swapaxes(X, 1, 2)
    Y_train = np.concatenate(Y_tr, axis = 0).astype(np.int)

    X = np.swapaxes(X_te, 1, 3)
    X_test = np.swapaxes(X, 1, 2).astype(np.float)
    Y_test = Y_te.astype(np.int)

    print('Done ! Loaded cifar10 dataset')
    return X_train/255, Y_train, X_test/255, Y_test

In [3]:
class Helper:
    
    def __init__(self):
        self.name = 'Abhishek Kumar'
        print('Abhishek is here to help')
        
    def sample_targets(self, N, Z):
        '''
        N : number of targets to be sampled
        Z : dimenstionality of unit sphere
        '''

        samples = np.random.normal(0,1, [N,Z]).astype(np.float32)
        magnitudes = np.expand_dims(np.sqrt(np.sum(np.square(samples),axis=1)),1)

        return samples/magnitudes

    def get_nearest_targets(self, samples, targets):
        print('Getting optimal assignments of labels..')
        to_optimize = np.zeros([samples.shape[0], targets.shape[0]])
        for i in range(samples.shape[0]):
            to_optimize[:,i] = np.sum(np.square(samples-targets[i,:]), axis = 1)

        _, ind_assigns = sopt.linear_sum_assignment(to_optimize)
        return targets[ind_assigns]

    def shuffle_unision(self, nd1, nd2):
        assert nd1.shape[0] == nd2.shape[0], 'Arrays first dimesnsions should of same size' 
        permutation = np.random.permutation(nd1.shape[0])
        return nd1[permutation], nd2[permutation]

    def get_collage(self, images, size):
        h, w = images.shape[1], images.shape[2]
        print(images.shape)
        img = np.zeros((h * size[0], w * size[1], 3))

        for idx, image in enumerate(images[0:size[0]*size[1], :, :, :]):
            i = idx % size[1]
            j = idx // size[1]
            img[j*h:j*h+h, i*w:i*w+w, :] = image
        return img

In [4]:
class FB_NAT:
    
    # https://github.com/mrjel/noise-as-targets-tensorflow/blob/master/model.py
    def __init__(self, args, input_shape = [32,32,3]):
        
        #Assuming for rgb image data
        W,H,C = input_shape
        self.input_shape = input_shape
        
        # Placeholders
        self.args = args
        self.input_ = tf.placeholder(dtype=tf.float32, shape=[None, W, H, C]) # input_train_ph
        self.target_ = tf.placeholder(dtype=tf.float32, shape=[None, self.args['latent_dim']]) # targets
        self.drop_rate_ = tf.placeholder(tf.float32) # dropout_keep_prob
        self.initial_lr_ = tf.placeholder(tf.float32) # new_lr
        
        # Variables
        self.lr = tf.Variable(self.args['lr'])
        
        # Preprocessings
        self.input_tr = self.preprocess(self.input_)
        
        # Model Definition
        self.latent_vec = self.encoder_network(self.input_tr)
        
        # Classifier
        self.c_labels_ = tf.placeholder(dtype=tf.int32, shape=[None], name = 'classfier_labels')
        self.initial_c_lr_ = tf.placeholder(tf.float32)
        self.c_lr = tf.Variable(self.args['c_lr'])
        
        self.logits_ = self.classifier(self.latent_vec)
        self.cla_top_k = tf.nn.in_top_k(self.logits_,self.c_labels_,1)
        
        
        # Objective Losses
        self.loss = self.get_loss()
        self.cla_loss = self.get_cla_loss()
        
        # Trainings
        self.train = self.train_ret()
        self.cla_train = self.cla_train_ret()
        
        # update lr
        self.update_lr = tf.assign(self.lr, self.initial_lr_, name='lr_update')
        self.update_lr_cla = tf.assign(self.c_lr, self.initial_c_lr_, name='cla_lr_update')
    
    
    def encoder_network(self, x, reuse_variables = False, d_format = 'channels_last'):
        with tf.variable_scope("enc", reuse = reuse_variables) as enc_vs:
            latent_dim = self.args['latent_dim']
            num_layers = 4
            hid_c = 64
            # Deep Network
            x = tf.layers.conv2d(x, hid_c, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c, 3, 2, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*2, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*2, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*2, 3, 2, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*3, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*3, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*3, 3, 2, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*4, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            x = tf.layers.conv2d(x, hid_c*4, 3, 1, padding='same', activation=tf.nn.elu, data_format=d_format)
            
            # Flattened layers
            x = tf.layers.flatten(x,name=None,data_format=d_format)
            x = tf.nn.dropout(x,self.drop_rate_)
            x = tf.layers.dense(x, hid_c*4, activation=tf.nn.elu)
            x = tf.nn.dropout(x,self.drop_rate_)
            x = tf.layers.dense(x, latent_dim, activation=None)
            z = tf.nn.l2_normalize(x,1)
            
        self.enc_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='enc')
        
        return z
    
    
    def classifier(self, x, reuse_variables = False):
        with tf.variable_scope("cla",reuse=reuse_variables) as cla_vs:

            x = tf.layers.dense(x, self.args['n_classes']*20, activation=tf.nn.relu)
            x = tf.layers.dense(x, self.args['n_classes']*20, activation=tf.nn.relu)
            x = tf.layers.dense(x, self.args['n_classes'], activation=None, name = 'classfier_logits')

        self.cla_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='cla')
        return x
        
    
    def preprocess(self, input_):
        
        # Augmenting the data
        W,H,C = self.input_shape
        
        input_ = tf.map_fn(lambda img: tf.image.flip_left_right(img), input_)
        input_ = tf.map_fn(lambda img: tf.image.random_brightness(img,max_delta=63), input_)
        input_ = tf.map_fn(lambda img: tf.image.random_contrast(img, lower=0.2, upper=1.8),input_)
        input_ = tf.map_fn(lambda img: tf.image.per_image_standardization(img),input_)
        input_ = tf.map_fn(lambda img: tf.image.resize_image_with_crop_or_pad(img,int(H*(30/32)),int(H*(30/32))),input_)
        input_ = tf.map_fn(lambda img: tf.image.resize_image_with_crop_or_pad(img,int(W*(42/32)),int(H*(42/32))),input_)
        input_ = tf.map_fn(lambda img: tf.random_crop(img,[W,H,C]),input_)
        
        return input_
    
    
        
    
    def cla_train_ret(self):
        optimizer = tf.train.AdamOptimizer(self.c_lr)
        cla_train = optimizer.minimize(self.cla_loss, var_list = self.cla_vars)
        return cla_train
    
        
    def train_ret(self):
        optimizer = tf.train.AdamOptimizer(self.lr)
        enc_train = optimizer.minimize(self.loss, var_list = self.enc_vars)
        return enc_train
    
    def get_cla_loss(self):
        return tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits_,labels = self.c_labels_))
    
    def get_loss(self):
        return tf.reduce_mean(self.l2_sqr(self.target_, self.latent_vec))
    
    
    def l2_sqr(self, x, y):
        return tf.reduce_sum(tf.square(x - y),axis=1)
    
    def l1(self, x, y):
        return tf.reduce_sum(tf.abs(x - y),axis=1)

In [5]:
args = dict(
    input_type='cifar_10',
    n_epochs = 200,
    lr = 0.0001,
    c_lr = 0.001,
    batch_size = 256,
    model_dir = '../models/NAT_FB',
    data_dir = '../data/cifar-10-batches-py',
    print_interval = 2,
    c_train_interval = 10,
    c_epochs = 1,
    latent_dim = 32,
    n_classes = 10
)
helper = Helper()

Abhishek is here to help


In [6]:
X_train, Y_train, X_test, Y_test = load_cifar10(args['data_dir'])

Loading cifar10 Dataset...
Done ! Loaded cifar10 dataset


In [7]:
tf.reset_default_graph()
encoder = FB_NAT(args)

W0816 22:36:24.058589 140480208467776 deprecation.py:323] From /home/abhi/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:1514: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0816 22:36:24.193047 140480208467776 deprecation.py:323] From <ipython-input-4-d38d3f136652>:54: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0816 22:36:24.196677 140480208467776 deprecation.py:506] From /home/abhi/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the c

In [8]:
# Classifier init
cla_initializer = tf.variables_initializer(encoder.cla_vars)
enc_initializer = tf.variables_initializer(encoder.enc_vars)
global_initializer = tf.initializers.global_variables()

In [ ]:
# model saver 
saver = tf.train.Saver()

In [ ]:
target = helper.sample_targets(X_train.shape[0], args['latent_dim'])

In [ ]:
# Training The model
train_loss = []
cla_loss = []
data_train_X = X_train.copy()
loss = 0

In [ ]:
with tf.Session() as sess:
    sess.run(global_initializer)
    
    batch_size = args['batch_size']
    num_batches = X_train.shape[0]//batch_size
    curr_epoch = -1
    for counter in range(args['n_epochs']*num_batches):
        if(counter%num_batches == 0):
            batch_ind = 0
            curr_epoch += 1
            
            target, data_train_X = helper.shuffle_unision(target, data_train_X)
        
        start = time.time()
        
        # Getting the current set of bacthes 
        batch_x = data_train_X[batch_ind:batch_ind + batch_size]
        batch_y = target[batch_ind:batch_ind + batch_size]
        
        # Recalc the optimal assigments
        if(curr_epoch%3 == 0):            
            feed_dict_ = {encoder.input_:batch_x, encoder.drop_rate_:1.0}
            z = sess.run(encoder.latent_vec, feed_dict = feed_dict_)
            
            # Using Hungsess.run(tf.variables_initializer(encoder.enc_vars))arain Algorithm of Scipy
            batch_y = helper.get_nearest_targets(z, batch_y)
            target[batch_ind:batch_ind + batch_size] = batch_y
        
        # Training the network
        feed_dict_ = {encoder.target_: batch_y, encoder.input_: batch_x,encoder.drop_rate_:0.5}
        fetch_dict = {
                "train": encoder.train
        }
        
        if(counter%args['print_interval'] == 0):
            fetch_dict.update({
                "loss" : encoder.loss
            })
            
            train_loss.append(loss)
        
        result = sess.run(fetch_dict, feed_dict = feed_dict_)
        
        end = time.time()
        
        if(counter%args['print_interval'] == 0):
            
            clr(wait = True)
            loss = result['loss']
            print("Epoch: {}, Batch: {}/{}, Loss: {:.4f}". \
                  format(curr_epoch, counter/num_batches, num_batches, loss))
            
            print('Train_loss_5', train_loss[-5:], '\n Classifier_loss_5', cla_loss[-5:])
        
        # Training Classifier
        if(curr_epoch%args['c_train_interval'] == 0 and counter/num_batches == 0):
            # ReInitialize
            sess.run(cla_initializer)
            cla_loss = []
            
            # Compute Vecs
            cal_batch_ind = 0
            perm = np.random.permutation(X_train.shape[0])
#             labels, inputs = helper.shuffle_unision(Y_train, X_train)
            
            for i in range(args['c_epochs']*num_batches):
                
                if(i%num_batches == 0):
                    i = 0
            
                batch_labels = Y_train[perm[i*batch_size:(i+1)*batch_size]]
                batch_inputs = X_train[perm[i*batch_size:(i+1)*batch_size]]
                
                _, loss, top_k = sess.run(
                    [encoder.cla_train,encoder.cla_loss, encoder.cla_top_k],
                         feed_dict={encoder.c_labels_:batch_labels, encoder.input_: batch_inputs,
                                    encoder.drop_rate_:1.0})
                
                top_k = np.sum(top_k)
                
                cla_loss.append(loss)
                
                if i % 30 == 0:
                    print("Classifier: Batch {}/{}, Loss: {:.4f}, Accuracy: {:.4f}". \
                              format(i,num_batches, loss, float(top_k)/batch_size))
                
                
                
        counter += 1
        batch_ind += batch_size

Epoch: 0, Batch: 0.6051282051282051/195, Loss: 1.4589
Train_loss_5 [1.460522, 1.4326866, 1.4792104, 1.444942, 1.4644113] 
 Classifier_loss_5 [1.9893492, 1.996409, 2.021879, 1.9884981, 1.9273882]
Getting optimal assignments of labels..
